In [70]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

We will create a MineSweeper game with a board size of (a,a). The game will be implemented using two matrix of size (a,a), which will be named the Game board and the Mine board. 

The Game board will be a matrix with numbers ranging from -2 to 8, where -1 represent a bomb, @ represent an uncover cell, empty space represent a cell without a bomb surrounding and 1-8 represent a cell with the number of bombs surrounding it. It will be the game board observed by the player.

The Mine Board will be a matrix with 0 and 1, where 1 is a bomb and 0 is a safe cell. This matrix will store the mine location on the board.

In [96]:
class minesweeper_game(object):
    def __init__(self, coord, len_board):
        self.len_board = len_board
        self.number_mines = len_board-1
        self.mine_board, self.game_board = self.create_boards(coord)
    def create_boards(self, coord): #We create both boards. We ask for coord so that the first input can not be a loss
        mine_board = np.full((self.len_board,self.len_board),0)  #We define a np.array of size (a,a) with only zeroes
        mines = [] #list of mines
        while len(mines) <= self.number_mines: #we want #self.len_board mines
            mine_coord = (random.randint(0, self.len_board-1), random.randint(0, self.len_board-1)) #we consider the coordinate of the mine
            if (coord != mine_coord) and (mine_coord not in mines): 
                mines.append(mine_coord)
                mine_board[mine_coord] = 1
        game_board = np.full((self.len_board,self.len_board),".") #game board is empty
        self.number_mines = len(mines)
        return mine_board, game_board
                    
    def game_status(self, coord): #Check the state of the game
        if self.mine_board[coord] == 1: #Uncovered mine = loss
            return 0
        uncovered_cells = self.len_board*self.len_board
        for i in range(0,self.len_board):
            for j in range(0, self.len_board):
                if (self.game_board[(i,j)] != "."):
                    uncovered_cells -= 1
        if uncovered_cells == self.number_mines:
            return 1 #You uncovered all of the cells without a bomb (There are "a" bombs).
        else:
            return 2 #The game is still going
    
    def update_game_board(self, coord): #After the player gives a coordinates, update the map with the new info
        i = coord[0]
        j = coord[1]
        if self.mine_board[coord] == 1: #Bomb
            self.game_board[coord] = "*"
        else:
            count_mines = 0
            #We have to separate the cases on which (a,b) it is in the border because we have to add the surrounding number of bombs
            if i == 0: #First row
                for m in range(0, 2): 
                    if j == 0: #Corner (0,0)
                        for n in range(0, 2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #Corner (0,len)
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else: #row x = 0 without corners
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            elif i == self.len_board-1: #Last row
                for m in range(i-1, i+1):
                    if j == 0: #corner (len, 0)
                        for n in range(j, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #corner (len, len)
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else: #row x = len without corners
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            else: #Not in first or last row
                for m in range(i-1, i+2):
                    if j == 0: #First column
                        for n in range(j, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #Last column
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else:  #points is in the middle
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            if count_mines == 0: #No mines = empty
                self.game_board[coord] = "@"
            else:
                self.game_board[coord] = count_mines #Number of mines surrounding it

    def uncover_all_empty_spaces(self): #If the coordinate added was a "@", we clear all "@" and number surrounding it.
        #We want to consider only the cells which are above, below and in the sides of a "@".
        covered_empty = True
        local_game_board = np.copy(self.game_board) #Copy of the original board, to compare
        while covered_empty == True:
            for i in range(0,self.len_board): #Rows
                for j in range(0,self.len_board): #Columns
                    if self.game_board[(i,j)] == "@":
                        if i == 0:
                            for m in range(0, 2): #we can only take row 0 and 1
                                if j == 0: #Corner (0,0)
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,1))): 
                                            self.update_game_board((m,n))
                                elif (j == self.len_board-1): #Corner (0,len)
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,self.len_board-2))):
                                            self.update_game_board((m,n))
                                else: #row x = 0 without corners
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,j-1)) and ( (m,n) != (1,j+1))):
                                            self.update_game_board((m,n))
                        elif i == self.len_board-1:
                            for m in range(self.len_board-2, self.len_board):
                                if j == 0: #corner (len, 0)
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (self.len_board-2, 1))):
                                            self.update_game_board((m,n))
                                elif j == self.len_board-1: #corner (len, len)
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (self.len_board-2, self.len_board-2) )):
                                            self.update_game_board((m,n))
                                else: #row x = len without corners
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, j-1)) and ( (m,n) != (i-1,j+1))) :
                                            self.update_game_board((m,n))
                        else:
                            for m in range(i-1, i+2):
                                if j == 0: #First column
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1,1) ) and ((m,n) != (i+1,1))):
                                            self.update_game_board((m,n))
                                elif j == self.len_board-1: #Last column
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, self.len_board-2) ) and ( (m,n) != (i+1, self.len_board-2))):
                                            self.update_game_board((m,n))
                                else: #coordinate is in the middle
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, j-1) ) and ((m,n) != (i-1, j+1) ) and ((m,n) != (i+1, j-1) ) and ((m,n) != (i+1, j+1) )):
                                            self.update_game_board((m,n))
                    else:
                        continue
                else:
                    continue
            if np.array_equal(local_game_board, self.game_board):
                #To stop we check that the last loop didnt added a new cleared element.
                covered_empty = False
            else:
                local_game_board = np.copy(self.game_board) #If we added new cells, we update the local_game_board to compare again after the loop.
#Helpful functions:
def input_int(): #To check that the input is the desire type
    Not_integer = True
    while Not_integer == True:
        try:
            len_board = int(input("Enter a number: "))
            return int(len_board)
        except ValueError:
            print("Input is not an integer")
    
def input_coord(): #To check that the input is the desire type
    Not_tuple = True
    while Not_tuple == True:
        try:
            coord = input("Enter a tuple of the form a,b: ").split(",")
            i = coord[0]
            j = coord[1]
            i = int(i)
            j = int(j)
            return (i,j)
        except ValueError:
            print("Input is not a tuple of two integers")
    
            
      

In [97]:
print("Minesweeper!. First, introduce the length of the board (an integer):")
game_status = 2
len_board = input_int()
print("The following board is your game board. It is a matrix with numbers ranging from -2 to 8, where -1 represent a bomb,\n @ represent an uncover cell, and 0-8 represent a cell with the number of bombs surrounding it")
print(np.full((len_board,len_board), "."))
print("Introduce the coordinates of the cell you want to reveal. This has to be a tuple of the form (a,b).")
coord = input_coord()
game = minesweeper_game(coord, len_board)
game.update_game_board(coord)
game.uncover_all_empty_spaces()
print(game.game_board)
while game_status == 2:
    print("Introduce the coordinates of the following cell you want to reveal:")
    coord = input_coord()
    game.update_game_board(coord)
    game.uncover_all_empty_spaces()
    print(game.game_board)
    game_status = game.game_status(coord)
if game_status == 0:
    print("You lost!")
if game_status == 1:
    print("You won!")    

Minesweeper!. First, introduce the length of the board (an integer):
The following board is your game board. It is a matrix with numbers ranging from -2 to 8, where -1 represent a bomb,
 @ represent an uncover cell, and 0-8 represent a cell with the number of bombs surrounding it
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Introduce the coordinates of the cell you want to reveal. This has to be a tuple of the form (a,b).
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '2' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Introduce the coordinates of the following cell you want 